# Structured Output Parser

- Author: [Yoolim Han](https://github.com/hohosznta)
- Design: []()
- Peer Review: []()
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/sub-graph.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239937-lesson-2-sub-graphs)

## Overview

The `StructuredOutputParser` is a valuable tool for formatting Large Language Model (LLM) responses into dictionary structures, enabling the return of multiple fields as key/value pairs. 
hile Pydantic and JSON parsers offer robust capabilities, the `StructuredOutputParser `is particularly effective for less powerful models, such as local models with fewer parameters. It is especially beneficial for models with lower intelligence compared to advanced models like GPT or Claude. 
By utilizing the `StructuredOutputParser`, developers can maintain data integrity and consistency across various LLM applications, even when operating with models that have reduced parameter counts.

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Implementing Structured Output Parser](#implementing-structured-output-parser)

### References

- [LangChain ChatOpenAI API reference](https://python.langchain.com/docs/integrations/chat/openai/)
- [LangChain Structured output parser](https://python.langchain.com/docs/how_to/output_parser_structured/)
---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

'DOSKEY'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
        "langchain_community",
    ],
    verbose=False,
    upgrade=False,
)


In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "03-StructuredOutputParser",
    }
)


Environment variables have been set successfully.


You can alternatively set `OPENAI_API_KEY` in `.env` file and load it. 

[Note] This is not necessary if you've already set `OPENAI_API_KEY` in previous steps.

In [4]:
from dotenv import load_dotenv

load_dotenv(override=True)


False

## Implementing Structured Output Parser

### Using ResponseSchema with StructuredOutputParser
*   Define a response schema using the ResponseSchema class to include the answer to the user's question and a description of the source (website) used.

*   Initialize `StructuredOutputParser` with response_schemas to structure the output according to the defined response schema.

**[Note]**
When using local models, Pydantic parsers may frequently fail to work properly. In such cases, using `StructuredOutputParser` can be a good alternative solution.

In [5]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# Response to the user's question
response_schemas = [
    ResponseSchema(name="answer", description="Answer to the user's question"),
    ResponseSchema(
        name="source",
        description="The `source` used to answer the user's question, which should be a `website URL`.",
    ),
]
# Initialize the structured output parser based on the response schemas
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


### Embedding Response Schemas into Prompts 

Create a PromptTemplate to format user questions and embed parsing instructions for structured outputs.

In [6]:
from langchain_core.prompts import PromptTemplate
# Parse the format instructions.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # Set up the template to answer the user's question as best as possible.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # Use 'question' as the input variable.
    input_variables=["question"],
    # Use 'format_instructions' as a partial variable.
    partial_variables={"format_instructions": format_instructions},
)

### Integrating with ChatOpenAI and Running the Chain

Combine the `PromptTemplate`, `ChatOpenAI` model, and `StructuredOutputParser` into a chain. Finally, run the chain with a specific `question` to produce results.

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)  # Initialize the ChatOpenAI model

chain = prompt | model | output_parser  # Connect the prompt, model, and output parser

# What is the largest desert in the world?
chain.invoke({"question": "What is the largest desert in the world?"})

c:\Users\HP\anaconda3\envs\llama3phidata\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=ffe4d59c-bc75-4f4d-a5fc-5f85ccfe709c,id=ffe4d59c-bc75-4f4d-a5fc-5f85ccfe709c; trace=ffe4d59c-bc75-4f4d-a5fc-5f85ccfe709c,id=8eb08352-4d72-442b-9dad-274c9fda10e1
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=ffe4d59c-bc75-4f4d-a5fc-5f85ccfe709c,id=685effb4-a09b-4b4d-a5d6-e20e3473554f; trace=ffe4d59c-b

{'answer': 'The largest desert in the world is the Antarctic Desert.',
 'source': 'https://www.britannica.com/place/desert'}

### Using Streamed Outputs

Use the `chain.stream` method to receive a streaming response to the question, "What are the achievements of King Sejong?"

In [8]:
for s in chain.stream({"question": "How many players are on a soccer team?"}):
    # Stream the output
    print(s)

Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Invalid token"}')trace=ffe4d59c-bc75-4f4d-a5fc-5f85ccfe709c,id=7489538d-e459-4594-a59c-c6e2ae5b7cb1; trace=b68f99d1-3b0e-4332-84d8-34e2ee45a689,id=b68f99d1-3b0e-4332-84d8-34e2ee45a689; trace=b68f99d1-3b0e-4332-84d8-34e2ee45a689,id=76c8bfe4-359f-45bc-9855-c37b7e5974d3; trace=b68f99d1-3b0e-4332-84d8-34e2ee45a689,id=c3635d6b-fbd0-412f-96c5-2ec496911075; trace=ffe4d59c-bc75-4f4d-a5fc-5f85ccfe709c,id=ffe4d59c-bc75-4f4d-a5fc-5f85ccfe709c; trace=ffe4d59c-bc75-4f4d-a5fc-5f85ccfe709c,id=685effb4-a09b-4b4d-a5d6-e20e3473554f
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com

{'answer': 'A standard soccer team consists of 11 players on the field at a time.', 'source': 'https://www.fifa.com/who-we-are/news/what-are-the-rules-of-football-2040008'}
